In [1]:
# from backtesting import Backtest, Strategy
# from backtesting.lib import crossover
# from backtesting.test import SMA, GOOG
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
# from xgboost import XGBClassifier
# import datetime
# from sktime.classification.all import *
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, make_scorer, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
# import lightgbm as lgbm
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.utils import to_categorical
# from keras.utils import plot_model
# from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras import regularizers
# import plotly.graph_objects as go
# import ta
import pandas_ta as ta

In [2]:
num_diff = 365*10
current_date = datetime.now()
time1 = (current_date + timedelta(days = - num_diff)).strftime("%Y-%m-%d")
time2 = current_date.strftime("%Y-%m-%d")
print("time1:", time1, "time2:", time2)
current_date = current_date + timedelta(days = - num_diff - 1)
df = yf.download("GC=F", start=time1, end=time2, interval="1d")
df['ret'] = df['Adj Close'].pct_change()
df['vol'] = df['ret'].rolling(20).std() * np.sqrt(252)
df['ret_fwd'] = df['Adj Close'].pct_change(-1)
df['ret_fwd3'] = df['Adj Close'].pct_change(-3)
df['ret_fwd5'] = df['Adj Close'].pct_change(-5)
df['ret_fwd10'] = df['Adj Close'].pct_change(-10)


df['class'] = df['ret_fwd'].apply(lambda x: 1 if x >= 0.005 else 0)
df['class3'] = df['ret_fwd3'].apply(lambda x: 1 if x >= 0.005 else 0)
df['class5'] = df['ret_fwd5'].apply(lambda x: 1 if x >= 0.005 else 0)
df['class10'] = df['ret_fwd10'].apply(lambda x: 1 if x >= 0.005 else 0)
# df['hammer'] = ta.CandlePatterns(df['Open'], df['High'], df['Low'], df['Close']).cdl_hammer()

df = df.drop('Close', axis=1).rename(columns={'Adj Close': 'Close'})

for i in range(1,(3*22)):
    df['Open_L' + str(i)] = df['Open'].shift(i)
    df['Close_L' + str(i)] = df['Close'].shift(i)
    df['High_L' + str(i)] = df['High'].shift(i)
    df['Low_L' + str(i)] = df['Low'].shift(i)

# df[['is_bullish_engulfing', 'is_bearish_engulfing', 'is_bullish_harami', 'is_bearlish_harami', 'is_bullish_hammer', 'is_bearlish_hammer']] = df[['Open', 'Close', 'High', 'Low', 'Open_L1', 'Close_L1', 'High_L1', 'Low_L1']].apply(get_candle_stick_pattern, axis=1, result_type="expand")
# df = df.dropna()
df

time1: 2013-10-05 time2: 2023-10-03
[*********************100%%**********************]  1 of 1 completed


C:\Users\kamin\AppData\Local\Temp\ipykernel_34664\2432718805.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['High_L' + str(i)] = df['High'].shift(i)
C:\Users\kamin\AppData\Local\Temp\ipykernel_34664\2432718805.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Low_L' + str(i)] = df['Low'].shift(i)
C:\Users\kamin\AppData\Local\Temp\ipykernel_34664\2432718805.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi

,Open,High,Low,Close,Volume,ret,vol,ret_fwd,ret_fwd3,ret_fwd5,...,High_L63,Low_L63,Open_L64,Close_L64,High_L64,Low_L64,Open_L65,Close_L65,High_L65,Low_L65
Date,,,,,,,,,,,,,,,,,,,,,
2013-10-07,1313.800049,1325.400024,1311.800049,1324.800049,84,NaN,NaN,0.000453,0.021749,0.037919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-08,1326.500000,1330.000000,1321.199951,1324.199951,57,-0.000453,NaN,0.013237,0.044322,0.040220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-09,1317.599976,1319.000000,1295.300049,1306.900024,140,-0.013064,NaN,0.007944,0.023895,0.019423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-10,1303.400024,1310.800049,1285.000000,1296.599976,89,-0.007881,NaN,0.022555,0.018539,-0.019732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-11,1286.900024,1291.599976,1263.800049,1268.000000,210,-0.022058,NaN,-0.006581,-0.010920,-0.035301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-26,1905.000000,1905.099976,1900.400024,1900.400024,161,-0.008452,0.091804,0.015388,0.028299,NaN,...,1929.400024,1914.000000,1922.900024,1923.699951,1928.500000,1922.500000,1918.699951,1919.099976,1932.500000,1918.699951
2023-09-27,1871.599976,1871.599976,1871.599976,1871.599976,12251,-0.015155,0.097598,0.006020,0.022732,NaN,...,1912.300049,1912.300049,1927.000000,1914.000000,1929.400024,1914.000000,1922.900024,1923.699951,1928.500000,1922.500000
2023-09-28,1875.500000,1878.199951,1856.699951,1860.400024,4654,-0.005984,0.096287,0.006656,NaN,NaN,...,1909.900024,1892.500000,1912.300049,1912.300049,1912.300049,1912.300049,1927.000000,1914.000000,1929.400024,1914.000000


In [3]:
features = ['Open', 'High', 'Low', 'Close']
features = features + ['Open_L' + str(i) for i in range(1,(3*22))] + ['Close_L' + str(i) for i in range(1,(3*22))] + ['High_L' + str(i) for i in range(1,(3*22))]+ ['Low_L' + str(i) for i in range(1,(3*22))]

# features += ['Open_L1d', 'High_L1d', 'Low_L1d', 'Close_L1d',
#              'Open_L2d', 'High_L2d', 'Low_L2d', 'Close_L2d',
#              'Open_L3d', 'High_L3d', 'Low_L3d', 'Close_L3d']

len(features)

264

In [4]:
all_indicators = list(df.ta.cdl_pattern(name="all").columns)
all_indicators

['CDL_2CROWS',
 'CDL_3BLACKCROWS',
 'CDL_3INSIDE',
 'CDL_3LINESTRIKE',
 'CDL_3OUTSIDE',
 'CDL_3STARSINSOUTH',
 'CDL_3WHITESOLDIERS',
 'CDL_ABANDONEDBABY',
 'CDL_ADVANCEBLOCK',
 'CDL_BELTHOLD',
 'CDL_BREAKAWAY',
 'CDL_CLOSINGMARUBOZU',
 'CDL_CONCEALBABYSWALL',
 'CDL_COUNTERATTACK',
 'CDL_DARKCLOUDCOVER',
 'CDL_DOJI_10_0.1',
 'CDL_DOJISTAR',
 'CDL_DRAGONFLYDOJI',
 'CDL_ENGULFING',
 'CDL_EVENINGDOJISTAR',
 'CDL_EVENINGSTAR',
 'CDL_GAPSIDESIDEWHITE',
 'CDL_GRAVESTONEDOJI',
 'CDL_HAMMER',
 'CDL_HANGINGMAN',
 'CDL_HARAMI',
 'CDL_HARAMICROSS',
 'CDL_HIGHWAVE',
 'CDL_HIKKAKE',
 'CDL_HIKKAKEMOD',
 'CDL_HOMINGPIGEON',
 'CDL_IDENTICAL3CROWS',
 'CDL_INNECK',
 'CDL_INSIDE',
 'CDL_INVERTEDHAMMER',
 'CDL_KICKING',
 'CDL_KICKINGBYLENGTH',
 'CDL_LADDERBOTTOM',
 'CDL_LONGLEGGEDDOJI',
 'CDL_LONGLINE',
 'CDL_MARUBOZU',
 'CDL_MATCHINGLOW',
 'CDL_MATHOLD',
 'CDL_MORNINGDOJISTAR',
 'CDL_MORNINGSTAR',
 'CDL_ONNECK',
 'CDL_PIERCING',
 'CDL_RICKSHAWMAN',
 'CDL_RISEFALL3METHODS',
 'CDL_SEPARATINGLINES',
 'CDL_SH

In [5]:
df

,Open,High,Low,Close,Volume,ret,vol,ret_fwd,ret_fwd3,ret_fwd5,...,High_L63,Low_L63,Open_L64,Close_L64,High_L64,Low_L64,Open_L65,Close_L65,High_L65,Low_L65
Date,,,,,,,,,,,,,,,,,,,,,
2013-10-07,1313.800049,1325.400024,1311.800049,1324.800049,84,NaN,NaN,0.000453,0.021749,0.037919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-08,1326.500000,1330.000000,1321.199951,1324.199951,57,-0.000453,NaN,0.013237,0.044322,0.040220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-09,1317.599976,1319.000000,1295.300049,1306.900024,140,-0.013064,NaN,0.007944,0.023895,0.019423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-10,1303.400024,1310.800049,1285.000000,1296.599976,89,-0.007881,NaN,0.022555,0.018539,-0.019732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-11,1286.900024,1291.599976,1263.800049,1268.000000,210,-0.022058,NaN,-0.006581,-0.010920,-0.035301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-26,1905.000000,1905.099976,1900.400024,1900.400024,161,-0.008452,0.091804,0.015388,0.028299,NaN,...,1929.400024,1914.000000,1922.900024,1923.699951,1928.500000,1922.500000,1918.699951,1919.099976,1932.500000,1918.699951
2023-09-27,1871.599976,1871.599976,1871.599976,1871.599976,12251,-0.015155,0.097598,0.006020,0.022732,NaN,...,1912.300049,1912.300049,1927.000000,1914.000000,1929.400024,1914.000000,1922.900024,1923.699951,1928.500000,1922.500000
2023-09-28,1875.500000,1878.199951,1856.699951,1860.400024,4654,-0.005984,0.096287,0.006656,NaN,NaN,...,1909.900024,1892.500000,1912.300049,1912.300049,1912.300049,1912.300049,1927.000000,1914.000000,1929.400024,1914.000000


In [6]:
df = pd.concat([df, df.ta.cdl_pattern(name="all")], axis=1)
df

,Open,High,Low,Close,Volume,ret,vol,ret_fwd,ret_fwd3,ret_fwd5,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
Date,,,,,,,,,,,,,,,,,,,,,
2013-10-07,1313.800049,1325.400024,1311.800049,1324.800049,84,NaN,NaN,0.000453,0.021749,0.037919,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-10-08,1326.500000,1330.000000,1321.199951,1324.199951,57,-0.000453,NaN,0.013237,0.044322,0.040220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-10-09,1317.599976,1319.000000,1295.300049,1306.900024,140,-0.013064,NaN,0.007944,0.023895,0.019423,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-10-10,1303.400024,1310.800049,1285.000000,1296.599976,89,-0.007881,NaN,0.022555,0.018539,-0.019732,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-10-11,1286.900024,1291.599976,1263.800049,1268.000000,210,-0.022058,NaN,-0.006581,-0.010920,-0.035301,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-26,1905.000000,1905.099976,1900.400024,1900.400024,161,-0.008452,0.091804,0.015388,0.028299,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-09-27,1871.599976,1871.599976,1871.599976,1871.599976,12251,-0.015155,0.097598,0.006020,0.022732,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-09-28,1875.500000,1878.199951,1856.699951,1860.400024,4654,-0.005984,0.096287,0.006656,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df.loc[:,features] =  (df.loc[:,features].T / df['Close_L' + str(3*22 - 1)]).T

In [8]:
df = df.dropna()

In [9]:
useable_indicators = []
for i in all_indicators:
    if len(df[df[i]!=0]) >= 30:
        useable_indicators.append(i)
#     break

In [10]:
useable_indicators

['CDL_3INSIDE',
 'CDL_3OUTSIDE',
 'CDL_BELTHOLD',
 'CDL_CLOSINGMARUBOZU',
 'CDL_DOJI_10_0.1',
 'CDL_DOJISTAR',
 'CDL_DRAGONFLYDOJI',
 'CDL_ENGULFING',
 'CDL_GRAVESTONEDOJI',
 'CDL_HAMMER',
 'CDL_HANGINGMAN',
 'CDL_HARAMI',
 'CDL_HARAMICROSS',
 'CDL_HIGHWAVE',
 'CDL_HIKKAKE',
 'CDL_HOMINGPIGEON',
 'CDL_INSIDE',
 'CDL_INVERTEDHAMMER',
 'CDL_LONGLEGGEDDOJI',
 'CDL_LONGLINE',
 'CDL_MARUBOZU',
 'CDL_MATCHINGLOW',
 'CDL_RICKSHAWMAN',
 'CDL_SHOOTINGSTAR',
 'CDL_SHORTLINE',
 'CDL_SPINNINGTOP',
 'CDL_TAKURI']

In [11]:
df[df[i]!=0][i]

Date
2014-10-08    100.0
2015-04-10   -100.0
2015-04-15   -100.0
2015-05-04   -100.0
2015-08-17   -100.0
2015-12-24   -100.0
2016-12-07   -100.0
2019-02-11    100.0
2020-02-11    100.0
2021-04-13   -100.0
2022-05-31    100.0
2023-05-05    100.0
Name: CDL_XSIDEGAP3METHODS, dtype: float64

In [12]:
for i in useable_indicators:
    all_values = df[i].unique()
    tmp_df = df[[i, 'ret_fwd', 'ret_fwd3', 'ret_fwd5']]
    tmp_mean = tmp_df.groupby(i, as_index=False).mean().set_index(i)
    tmp_n = tmp_df.groupby(i, as_index=False).count().set_index(i)
    tmp_std = tmp_df.groupby(i, as_index=False).std().set_index(i)
    tmp_se = tmp_std/np.sqrt(tmp_n)
    tmp_t = tmp_mean/tmp_se
    tmp_mean2 = tmp_mean.where(tmp_t.abs() > 1.96, '-')
    if np.max(np.abs(tmp_t)) >= 1.96:
        features.append(i)
        print(i)
        print("max_ret:", tmp_mean.max().max(), "min_ret:", tmp_mean.min().min())
        display(tmp_mean2)
        display(tmp_n)
        display(tmp_t)
        print()
        print('--------------------------------------------')
#     break

CDL_BELTHOLD
max_ret: 0.0003904234488518127 min_ret: -0.003101211491873989


,ret_fwd,ret_fwd3,ret_fwd5
CDL_BELTHOLD,,,
-100.0,-,-,-0.003101
0.0,-,-,-
100.0,-,-,-


,ret_fwd,ret_fwd3,ret_fwd5
CDL_BELTHOLD,,,
-100.0,263,263,263
0.0,1863,1863,1863
100.0,310,310,310


,ret_fwd,ret_fwd3,ret_fwd5
CDL_BELTHOLD,,,
-100.0,-0.482417,-1.075173,-2.644051
0.0,-0.992989,-1.074741,-0.808843
100.0,0.847940,-0.155205,-0.508456



--------------------------------------------
CDL_DRAGONFLYDOJI
max_ret: 0.0028281195385986744 min_ret: -0.0007598017819561329


...


--------------------------------------------
CDL_TAKURI
max_ret: 0.0023560358607636043 min_ret: -0.0007750573121045885


,ret_fwd,ret_fwd3,ret_fwd5
CDL_TAKURI,,,
0.0,-,-,-
100.0,0.002356,-,-


,ret_fwd,ret_fwd3,ret_fwd5
CDL_TAKURI,,,
0.0,2353,2353,2353
100.0,83,83,83


,ret_fwd,ret_fwd3,ret_fwd5
CDL_TAKURI,,,
0.0,-1.224158,-1.390180,-1.863904
100.0,2.170048,0.115156,0.667372



--------------------------------------------


In [6]:
CustomStrategy = ta.Strategy(
    name="Momo and Volatility",
    description="SMA 50,200, BBANDS, RSI, MACD and Volume SMA 20",
    ta=[
        {"kind": "sma", "length": 50},
        {"kind": "sma", "length": 200},
        {"kind": "bbands", "length": 20},
        {"kind": "rsi"},
        {"kind": "macd", "fast": 8, "slow": 21},
        {"kind": "sma", "close": "volume", "length": 20, "prefix": "VOLUME"},
    ]
)
# To run your "Custom Strategy"
k = df.ta.strategy(CustomStrategy)

In [7]:
k

In [8]:
df

,Open,High,Low,Close,Volume,ret,vol,ret_fwd,ret_fwd3,ret_fwd5,...,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,RSI_14,MACD_8_21_9,MACDh_8_21_9,MACDs_8_21_9,VOLUME_SMA_20
Date,,,,,,,,,,,,,,,,,,,,,
2013-10-03,NaN,NaN,NaN,NaN,217,NaN,NaN,0.005879,-0.005135,0.016042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-04,NaN,NaN,NaN,NaN,38,-0.005845,NaN,-0.011398,0.002142,0.032886,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-07,NaN,NaN,NaN,NaN,84,0.011529,NaN,0.000453,0.021749,0.037919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-08,NaN,NaN,NaN,NaN,57,-0.000453,NaN,0.013237,0.044322,0.040220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-09,NaN,NaN,NaN,NaN,140,-0.013064,NaN,0.007944,0.023895,0.019423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-25,1.002039,1.002039,1.002039,1.002039,114,-0.004571,0.087713,0.008524,0.030209,NaN,...,0.968264,0.986155,1.004045,3.628287,0.943943,61.881248,0.007755,0.001933,0.005822,199.25
2023-09-26,0.992653,0.992705,0.990256,0.990256,161,-0.008452,0.091804,0.015388,0.028299,NaN,...,0.969336,0.986811,1.004287,3.541813,0.598561,54.234199,0.006849,0.000822,0.006028,202.35
2023-09-27,0.972917,0.972917,0.972917,0.972917,12251,-0.015155,0.097598,0.006020,NaN,NaN,...,0.967760,0.985627,1.003493,3.625427,0.144309,45.352675,0.003794,-0.001787,0.005581,731.20


In [9]:
def precision_focus(y_true, y_pred):
#     print(y_true)
#     print(y_pred)
    precisions = precision_score(y_true, y_pred, average=None)
    return (precisions[1])

In [16]:
day_tr = 365*2
day_te = 90
date_max_te = df.index[-1]
for i in range(10):
    date_min_te = date_max_te + timedelta(days = - day_te)
    date_max_tr = date_min_te #+ timedelta(days = - 1)
    date_min_tr = date_max_tr + timedelta(days = - day_tr)
    tmp_tr = df[(df.index >= date_min_tr)&(df.index <= date_max_tr)].copy()
    tmp_te = df[(df.index >= date_min_te)&(df.index <= date_max_te)].copy()
    print("date_min_tr:", date_min_tr, "date_max_tr:", date_max_tr)
    print("date_min_te:", date_min_te, "date_max_te:", date_max_te)
    print('--------------------------------------')
    date_max_te = date_max_te + timedelta(days = - day_te)

    break

date_min_tr: 2021-07-01 00:00:00 date_max_tr: 2023-07-01 00:00:00
date_min_te: 2023-07-01 00:00:00 date_max_te: 2023-09-29 00:00:00
--------------------------------------


In [20]:
%%time
parameters = {'max_depth':[3, 10, 20, 40, 80, 120, None], 'n_estimators':[100]}
model = RandomForestClassifier(class_weight = 'balanced')
clt = GridSearchCV(model , parameters, cv = 3, scoring='precision')

clt.fit(tmp_tr[features], tmp_tr['class3'])
y_pred_tr = clt.predict(tmp_tr[features])
y_pred_te = clt.predict(tmp_te[features])
confusion_matrix(tmp_tr['class3'] , y_pred_tr)

CPU times: total: 4.66 s
Wall time: 14.6 s


array([[330,   0],
       [  0, 173]], dtype=int64)

In [21]:
confusion_matrix(tmp_te['class3'] , y_pred_te)

array([[37,  4],
       [19,  3]], dtype=int64)

In [15]:
tmp_tr[features]

,Open,High,Low,Close,Open_L1,Open_L2,Open_L3,Open_L4,Open_L5,Open_L6,...,Low_L56,Low_L57,Low_L58,Low_L59,Low_L60,Low_L61,Low_L62,Low_L63,Low_L64,Low_L65
Date,,,,,,,,,,,,,,,,,,,,,
2022-07-01,0.939069,0.944613,0.937029,0.940847,0.946444,0.950994,0.954079,0.957374,0.952144,0.959990,...,1.015586,1.008787,1.003923,1.002458,1.003347,1.001883,1.002615,1.003713,1.001308,0.990324
2022-07-05,0.933747,0.933747,0.911197,0.911197,0.928627,0.935919,0.940419,0.943470,0.946729,0.941557,...,1.008844,1.004293,0.997569,0.992759,0.991311,0.992190,0.990742,0.991466,0.992552,0.990173
2022-07-06,0.904217,0.907039,0.889596,0.890057,0.926226,0.921147,0.928381,0.932844,0.935871,0.939103,...,1.010415,1.000718,0.996204,0.989534,0.984763,0.983326,0.984199,0.982762,0.983480,0.984558
2022-07-07,0.909385,0.909385,0.905112,0.905581,0.918399,0.940754,0.935595,0.942942,0.947475,0.950550,...,1.022875,1.026262,1.016414,1.011828,1.005054,1.000208,0.998749,0.999635,0.998176,0.998906
2022-07-08,0.900943,0.906749,0.897833,0.902239,0.904624,0.913591,0.935828,0.930697,0.938005,0.942515,...,1.022600,1.017520,1.020890,1.011093,1.006531,0.999793,0.994972,0.993521,0.994402,0.992950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26,0.987723,0.990600,0.987518,0.988134,0.985566,0.986234,0.994247,1.006061,1.007499,0.997380,...,1.016540,1.001644,1.010890,1.004161,1.009914,1.013150,1.002877,1.018132,1.022447,1.000000
2023-06-27,0.966496,0.967700,0.959976,0.959976,0.964440,0.962333,0.962985,0.970809,0.982345,0.983750,...,1.009931,0.992577,0.978032,0.987060,0.980490,0.986107,0.989267,0.979236,0.994132,0.998345
2023-06-28,0.964785,0.964785,0.964785,0.964785,0.972201,0.970133,0.968014,0.968670,0.976540,0.988144,...,1.011553,1.015892,0.998436,0.983805,0.992886,0.986277,0.991928,0.995106,0.985016,1.000000
